In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!ls "/content/drive/My Drive/Web Mining Tut/Project/"
%cd /content/drive/My\ Drive/Web Mining Tut/Project

Mounted at /content/drive
data		     dict_clear.txt  graph_sampled.png	New_data.ipynb
Data_analysis.ipynb  dict.txt	     graph_test.ipynb	RiotMining.ipynb
/content/drive/My Drive/Web Mining Tut/Project


In [ ]:
import numpy as np
import pandas as pd
import re
import networkx as nx
from matplotlib import pyplot as plt

# Data

## Matches

In [175]:
# matches
matches = pd.read_csv('./data/match_playerPUUIDs_timestamp_ProPlayers.csv', header= None)
matches.columns = ['match_id','player1', 'player2', 'player3', 'player4', 'player5', 'player6', 'player7', 'player8', 'player9', 'player10', 'timestamp']

In [176]:
# get rid of brackets 
for col in matches:
    matches[col] = matches[col].astype(str).str.replace("[\]\{\}\'\(\)\ \[]","")
#matches.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [177]:
# replace nan strings with nan values
matches.replace('nan', np.nan, inplace=True)

In [178]:
# check for na values
print(matches.isna().sum())
#len(matches)

match_id       0
player1        0
player2        0
player3       70
player4      142
player5      146
player6      155
player7      157
player8      166
player9      166
player10     173
timestamp    173
dtype: int64


In [179]:
# timestamps that are null
correct = matches[matches['timestamp'].isnull()]
mask = correct[correct['timestamp'].isnull()].player10.isnull()
correct.loc[mask,['timestamp','player9']] = correct.loc[mask,['player9','timestamp']].values

correct2 = correct[correct['timestamp'].isnull()]
correct = correct[correct['timestamp'].notnull()]

correct2[correct2['timestamp'].isnull()].player8.isnull()
mask = correct2[correct2['timestamp'].isnull()].player8.isnull()
correct2.loc[mask,['timestamp','player7']] = correct2.loc[mask,['player7','timestamp']].values

correct3 = correct2[correct2['timestamp'].isnull()]
correct2 = correct2[correct2['timestamp'].notnull()]

correct3[correct3['timestamp'].isnull()].player7.isnull()
mask = correct3[correct3['timestamp'].isnull()].player7.isnull()
correct3.loc[mask,['timestamp','player6']] = correct3.loc[mask,['player6','timestamp']].values

correct4 = correct3[correct3['timestamp'].isnull()]
correct3 = correct3[correct3['timestamp'].notnull()]

correct4[correct4['timestamp'].isnull()].player6.isnull()
mask = correct4[correct4['timestamp'].isnull()].player6.isnull()
correct4.loc[mask,['timestamp','player5']] = correct4.loc[mask,['player5','timestamp']].values

correct5 = correct4[correct4['timestamp'].isnull()]
correct4 = correct4[correct4['timestamp'].notnull()]

correct5[correct5['timestamp'].isnull()].player5.isnull()
mask = correct5[correct5['timestamp'].isnull()].player5.isnull()
correct5.loc[mask,['timestamp','player4']] = correct5.loc[mask,['player4','timestamp']].values

correct6 = correct5[correct5['timestamp'].isnull()]
correct5 = correct5[correct5['timestamp'].notnull()]

correct6[correct6['timestamp'].isnull()].player4.isnull()
mask = correct6[correct6['timestamp'].isnull()].player4.isnull()
correct6.loc[mask,['timestamp','player3']] = correct6.loc[mask,['player3','timestamp']].values

correct7 = correct6[correct6['timestamp'].isnull()]
correct6 = correct6[correct6['timestamp'].notnull()]

correct7[correct7['timestamp'].isnull()].player3.isnull()
mask = correct7[correct7['timestamp'].isnull()].player3.isnull()
correct7.loc[mask,['timestamp','player2']] = correct7.loc[mask,['player2','timestamp']].values
correct7 = correct7[correct7['timestamp'].notnull()]

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [180]:
# check if all rows catched
#len(correct) + len(correct2) + len(correct3) + len(correct4) + len(correct5) + len(correct6) + len(correct7)

In [181]:
corrects = pd.concat([ correct, correct2, correct3, correct4, correct5, correct6, correct7])
corrects["timestamp"]= pd.to_datetime(corrects["timestamp"], format="%Y-%m-%dT%H:%M:%S")

In [182]:
# combine to one dataframe
matches["timestamp"]= pd.to_datetime(matches["timestamp"], format="%Y-%m-%dT%H:%M:%S")
new_matches = matches[matches['timestamp'].notna()]
new_matches = pd.concat([new_matches, correct, correct2, correct3, correct4, correct5, correct6, correct7])

In [183]:
# check 
#new_matches.head()
#matches[matches['match_id']=='5777687631']
#new_matches[new_matches['match_id']=='5838219052']

##Melted

In [184]:
# melt matches
matches_melted = (new_matches.melt(['match_id', 'timestamp'], 
                       var_name='player_number',
                       value_name='player_puuid'))
#len(matches_melted)

In [185]:
#remove empty cells (empty puuids, empty matches in general)
matches_melted['player_puuid'].replace(' ', np.nan, inplace=True)
matches_melted['player_puuid'].isna().sum()
matches_melted = matches_melted[matches_melted['player_puuid'].notna()]

In [186]:
#get rid of BOTs
#bots = matches_melted[matches_melted['player_puuid'].str.len()<5]
#len(bots)
matches_melted = matches_melted[matches_melted['player_puuid'].str.len()>5]
matches_melted.head()

,match_id,timestamp,player_number,player_puuid
0,5860242034,2022-05-07 13:30:38+00:00,player1,Enn_3VvikuDftsgroni-2b7bOtX7dNNG5PcRxXSMJFXZLP...
1,5860157180,2022-05-07 12:53:17+00:00,player1,0dzqDyAd3j7-v5dm7uqQTYMYPUXIshFVnc7w5p9QGWxnU5...
2,5860193059,2022-05-07 12:28:32+00:00,player1,HHIn3MsOsdbqZpxwkhOPUk9JNJu6_GBPoTLkFuZTgIuBhs...
3,5860101282,2022-05-07 11:56:54+00:00,player1,NtaeqFFIGCpibRu4nrmXmih6yJ2hBGkNyF6sf7QBBtTXlf...
4,5857587216,2022-05-05 18:52:11+00:00,player1,Kmtv1nNQhRKyXV-FqDIBEIO1WhPA5LPOjAvsnRldYlQcXv...


In [187]:
matches_melted.describe()

,match_id,timestamp,player_number,player_puuid
count,489076,489076,489076,489076
unique,38066,36796,10,106539
top,5860242034,2022-05-06 23:14:42+00:00,player1,OlfQQY21G1tZpUE9h3gCHFkTb3EaveilTZ-n84Zqpqr1dK...
freq,20,60,49009,176


In [188]:
# check for duplicates
matches_melted[matches_melted.duplicated(subset=['match_id','timestamp'])].value_counts()

match_id    timestamp                  player_number  player_puuid                                                                  
5859934604  2022-05-07 09:17:36+00:00  player3        XOwVkFMyxHTAGpRZWQiPY5QBYog1-F4JWe22PzlWSyXRum2S2VgVwMpdIRoyJykLqlog-EE0jpg7ug    2
5858233696  2022-05-06 07:44:05+00:00  player9        oRNqUl1ldmfd6Ntp8LJX6hUitRU233KM1U7GGuR2SuMVzSZXJ3X4J3jjESbq_eIjfPruJSXDXqG3Xw    2
5851600992  2022-05-01 18:30:59+00:00  player3        QXotLMsrB6NcWvtT43iaDGKuFfAQkH7BwsWAGGvQwwOrioTA83m1Rq-QGN0vLTCT5Eu96Rcf4K2G4Q    2
                                       player2        _kwOtewmZV2QBDta9A1sjkXgPXzMkjjWP8jYa2JH5fWROnaijjfMMKlSmIh8HInajrlfiD62GTmCJg    2
                                       player10       59ZfUqIv7R9nXDbFi3Tk2j99Q5VKsmJArwOoCPy-VhOXVeBXbNZuB_ioo_4BYt3D4_fax_jFA1fZpg    2
                                                                                                                                       ..
5854583022  2022-05-03 17:59:49+00:00  

In [189]:
# get rid of duplicates
pro_matches = matches_melted.drop_duplicates(subset=["match_id", "timestamp", "player_number"])
pro_matches.describe()

,match_id,timestamp,player_number,player_puuid
count,379646,379646,379646,379646
unique,38066,36796,10,106539
top,5860242034,2022-05-06 22:45:51+00:00,player1,OlfQQY21G1tZpUE9h3gCHFkTb3EaveilTZ-n84Zqpqr1dK...
freq,10,40,38066,109


## Ranks

In [190]:
# ranks
ranks = pd.read_csv('./data/fullyMinedSumonerPUUID_ProPlayers_Rank.csv', header= None)
ranks.columns = ['player_puuid','rank']

In [191]:
# check for na and null values
print(ranks.isna().sum())
print(ranks.isnull().sum())

player_puuid    0
rank            0
dtype: int64
player_puuid    0
rank            0
dtype: int64


In [192]:
# get rid of brackets 
for col in ranks:
    ranks[col] = ranks[col].astype(str).str.replace("[\]\{\}\'\(\)\[]","")
ranks.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


,player_puuid,rank
0,LKeh_TBiEFYyjz-B7uHdaS7O2jVtD_qtYvPTY9SqZKc5Cy...,Diamond
1,Enn_3VvikuDftsgroni-2b7bOtX7dNNG5PcRxXSMJFXZLP...,Master
2,izRO8r7lCyv37_5RE1RAs3yMQWcbWPPN0ByYPO78gq5Fnv...,Unknown
3,9JE9uBmsUmrXVR3-wq8IAiR_4n-7vdrBoSAvA_t_Ih4K9m...,Master
4,_fsin6Phz882tazkAKJ1ubSQbsw8OfuCzyr9HmVPckGGPH...,Master


In [193]:
# check for duplicates
ranks.describe()
ranks = ranks.drop_duplicates()

## Final dataset

In [200]:
# merge matches_melted and ranks
df = pro_matches.merge(ranks, on= 'player_puuid', how='left')
df.head()

,match_id,timestamp,player_number,player_puuid,rank
0,5860242034,2022-05-07 13:30:38+00:00,player1,Enn_3VvikuDftsgroni-2b7bOtX7dNNG5PcRxXSMJFXZLP...,Master
1,5860157180,2022-05-07 12:53:17+00:00,player1,0dzqDyAd3j7-v5dm7uqQTYMYPUXIshFVnc7w5p9QGWxnU5...,Diamond
2,5860193059,2022-05-07 12:28:32+00:00,player1,HHIn3MsOsdbqZpxwkhOPUk9JNJu6_GBPoTLkFuZTgIuBhs...,Unknown
3,5860101282,2022-05-07 11:56:54+00:00,player1,NtaeqFFIGCpibRu4nrmXmih6yJ2hBGkNyF6sf7QBBtTXlf...,Master
4,5857587216,2022-05-05 18:52:11+00:00,player1,Kmtv1nNQhRKyXV-FqDIBEIO1WhPA5LPOjAvsnRldYlQcXv...,Unknown


In [196]:
len(df)

379646

In [197]:
len(pro_matches)

379646

Questions to answer in Network Analysis:

- Why are they popular?
- Why has this node high degree centrality?
- Why this node has high proximity prestige?